# About this notebook
This notebook is an example of how to perform the following tasks:
* Read .csv files generated from Trackmate
* Crop out .tif files for division regions
* Export ROIs associated with divisions

## Python imports
The cell below deals with importing necessary packages and dependencies for running the notebook. You don't need to worry about what it does, it just needs to be run :)

In [1]:
from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.path as mpltPath
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import scipy.optimize as opt
from scipy.ndimage.filters import gaussian_filter

from glob import glob
from tqdm.notebook import tqdm
from tifffile import imread, TiffFile

from csbdeep.utils import Path, normalize
from csbdeep.io import save_tiff_imagej_compatible
from skimage.draw import polygon

from stardist import export_imagej_rois
from stardist.models import StarDist2D

import csv
from pathlib import Path

Using TensorFlow backend.


## File information
The below cell tells the code where to find the file(s) to analyse, and also any preferences you have for the analysis.

### File location
`base_dir` is the path containing the folder structure with outputs from the `Collated_process_up_to_Trackmate.ipynb` notebook and `Tracking_helper.ijm` ImageJ macro.

### Analysis settings
`drift_corrected` is a flag to tell the code whether the analysis was performed on drift-corrected data or not (where drift-correction was performed in `Collated_process_up_to_Trackmate.ipynb`. This should be `True` is drift correction was performed, otherwise it should be `False`.

`two_colour_analysis` is a flag to tell the code whether the analysis was performed on two-colour data or not. This should be set to `True` if it was, otherwise this should be `False`.

`tracking_channel` is the channel that tracking was performed on, in the case of two-colour datasets. For example, if tracking was performed on channel 1, this value should be set to `1` accordingly. Ignore this variable if you're working on a single-colour dataset.

In [2]:
base_dir = raw_file_path = r'.\data\two-colour data'

drift_corrected = True
two_colour_analysis = True
tracking_channel = 1

The next cell loads the file paths into the code. It shouldn't need changing!

From now on 'tracked' refers to the channel on which Trackmate was run, and 'untracked' refers to the second channel, if applicable.

In [3]:
# Load raw files
base_dir = Path(base_dir)
raw_dir = base_dir/f'raw data'
raw_files = sorted(Path(raw_dir).rglob('*.tif'))

# Load drift-corrected files
if drift_corrected:
    registered_dir = base_dir/f'registered data'
    registered_files = sorted(Path(registered_dir).rglob('*.tif'))
             
# Load Trackmate output
tracks_dir = base_dir/f'tracking results'
tracks_files = sorted(Path(tracks_dir).rglob('*.csv'))
                              
# Load Stardist results
stardist_dir = base_dir/f'stardist results'

if two_colour_analysis:
    stardist_1_dir = stardist_dir/f'channel 1'
    stardist_2_dir = stardist_dir/f'channel 2'

if not two_colour_analysis:
    stardist_files_tracked_python = sorted(Path(stardist_dir).rglob('*.npz'))
    stardist_files_tracked_imagej = sorted(Path(stardist_dir).rglob('*.zip'))
else:
    if tracking_channel==1:        
        stardist_files_tracked_python = sorted(Path(stardist_1_dir).rglob('*.npz'))
        stardist_files_tracked_imagej = sorted(Path(stardist_1_dir).rglob('*.zip'))
        stardist_files_untracked_python = sorted(Path(stardist_2_dir).rglob('*.npz'))
        stardist_files_untracked_imagej = sorted(Path(stardist_2_dir).rglob('*.zip'))
    elif tracking_channel==2:
        stardist_files_tracked_python = sorted(Path(stardist_2_dir).rglob('*.npz'))
        stardist_files_tracked_imagej = sorted(Path(stardist_2_dir).rglob('*.zip'))
        stardist_files_untracked_python = sorted(Path(stardist_1_dir).rglob('*.npz'))
        stardist_files_untracked_imagej = sorted(Path(stardist_1_dir).rglob('*.zip'))
    else:
        raise ValueError('Value assigned to tracking_channel is a baddy')
                              
# Create save directory
results_dir = base_dir/f'results'
results_dir.mkdir(exist_ok=True)

n_files_to_analyse = len(tracks_files)
print(f'There are {n_files_to_analyse} files to analyse')

print("Successfully created save directory")

There are 2 files to analyse
Successfully created save directory


## Retrieving matching files
This cell is another little helper for getting files. Just run it, accept it, and move on.

In [4]:
def get_matching_files(f, drift_corrected, two_colour_analysis):
    
    rois_python_untracked = None
    rois_imagej_untracked = None
    
    if drift_corrected:
        for file in registered_files:
            if str(f) in str(file):
                registered = file
                break
    else:
        for file in raw_files:
            if str(f) in str(file):
                registered = file
                break

    if not two_colour_analysis:
     
        for file in stardist_files_tracked_python:
            if str(f) in str(file):
                rois_python_tracked = file
                break
    
        for file in stardist_files_tracked_imagej:
            if str(f) in str(file):
                rois_imagej_tracked = file
                break
    
    else:
        
        for file in stardist_files_tracked_python:
            if str(f) in str(file):
                rois_python_tracked = file
                break
        for file in stardist_files_tracked_imagej:
            if str(f) in str(file):
                rois_imagej_tracked = file
                break
    
        for file in stardist_files_untracked_python:
            if str(f) in str(file):
                rois_python_untracked = file
                break
        for file in stardist_files_untracked_imagej:
            if str(f) in str(file):
                rois_imagej_untracked = file
                break
    
    
    for file in tracks_files:
        if str(f) in str(file):
            rois_trackmate = file
            break
            
    
    return registered, rois_python_tracked, rois_imagej_tracked, rois_python_untracked, rois_imagej_untracked, rois_trackmate
    

### Image pre-processing
This cell ensures that datasets are the correct dimensions and also performs normalisation for user-friendly visualisation later on.

In [5]:
def track_pre_process(image):
    T = imread(str(image))

    print(f"Normalizing each frame -> 'timelapse' is meant for plotting, use 'T' for further analysis", flush=True)
    timelapse = T
    timelapse = np.stack([normalize(frame, 1,99.8, axis=(-2,-1), clip=True) for frame in tqdm(timelapse)])

    if T.ndim == 3:
        axes = 'TYX'
        timelapse = timelapse[...,np.newaxis]
        timelapse = np.repeat(timelapse,3,axis=-1)
    elif T.ndim == 4:
        axes = 'TCYX'
        assert timelapse.shape[1] == 2
        timelapse = np.moveaxis(timelapse,1,0)
        timelapse = np.stack((*timelapse, np.zeros_like(timelapse[0])),axis=-1)
    else:
        raise ValueError("not supported")
        
    with TiffFile(str(image)) as _file:
        imagej_metadata = _file.imagej_metadata
        
    print(f"Timelapse has axes {axes.replace('C','')} with shape {timelapse.shape}")

    return timelapse, T, imagej_metadata, axes

### Load ROIs and tracks
This is the specific function that loads the Stardist ROIs and the indices of ROIs in the identified tracks into handy lists.

In [6]:
def load_rois_and_tracks(rois_trackmate, rois_python_tracked, rois_python_untracked=None):
    
    print(f"Loading tracked python polygons from {rois_python_tracked}")
    polygons_tracked = np.load(str(rois_python_tracked), allow_pickle=True)
    
    polygon_untracked = None
    
    if rois_python_untracked is not None:
        print(f"Loading untracked python polygons from {rois_python_untracked}")
        polygons_untracked = np.load(str(rois_python_untracked), allow_pickle=True)
        
    
    print(f"Loading ROIs per track from {rois_trackmate}")
    tracks = []
    with open(str(rois_trackmate)) as _f:
        csv_reader = csv.reader(_f, delimiter=',')
        for row in csv_reader:
            # parse FRAME_INDEX name as tuple to index into loaded python rois
            tracks.append([tuple(int(v)-1 for v in r.strip().split('_')) for r in row])
    print(f"Found {len(tracks)} tracks in dataset!")
    
    return polygons_tracked, polygons_untracked, tracks

### Polygon processing helper functions
These functions help with matching polygons in the untracked channel to the polygons present in tracks (only applicable to two-colour data)

In [7]:
def get_centroid(polygon):
    
    xvals = polygon[0]
    yvals = polygon[1]
    nvals = len(xvals)
    
    A = 0
    c_x = 0
    c_y = 0
    
    for i in range(0, nvals):
        j = i+1
        
        xi = xvals[i]
        yi = yvals[i]
        
        if(j==nvals):
            j=0
        xj = xvals[j]
        yj = yvals[j]
        
        cross_val = xi*yj - xj*yi
        A += cross_val/2
        
        c_x += (xi+xj)*cross_val
        c_y += (yi+yj)*cross_val
    
    c_x /= 6*A
    c_y /= 6*A
    
    return c_x, c_y

def polygons_to_centroids(polygons):
    coords = polygons['coord']
    centroids_map = {}
    
    for n in range(0, len(coords)):
        centroids = []
        for polygon in coords[n]:
            c_x, c_y = get_centroid(polygon)
            centroids.append([c_x, c_y])

        centroids_map[n] = centroids
    
    return centroids_map

### Track processing helper functions
You don't need to change these, but for reference here is a description of what each helper function does.

`get_rois_for_track` loops through each frame of a track and associates the ROIs within each frame with Stardist polygon objects.

`get_box_for_rois` finds the bounding box to contain a tracked event so that it can be cropped out later.

`translate_rois` translates the coordinates of the Stardist ROIs from the whole image to a cropped image.

`_plot_polygon` draws the segmentations onto images for visual inspection.

`plot_track` plots each cropped frame in the track into a single image, with the Stardist segmentations drawn over the top.

`export_crop_with_rois` saves each cropped track as a separate tif stack, along with a .npz file of Python-readable Stardist ROIs for each track, ImageJ-readable .zip file of ROIs for each track, and a binary mask image of the relevant segmentations in each track.

In [10]:
def get_rois_for_track(track, polygons_tracked, polygons_untracked=None):
    # get coordinates for polygons in tracked channel
    coords_tracked = polygons_tracked['coord']
    
    # get coordinates and centrois for polygons in untracked channel
    if polygons_untracked is not None:
        coords_untracked = polygons_untracked['coord']
        centroids_untracked = polygons_to_centroids(polygons_untracked)
        
    track_rois = []
    track_maps = {}
    
    for roi in track:
        frame = roi[0]
        index_tracked_polygon = roi[1]
                
        assert frame < len(coords_tracked)
        assert index_tracked_polygon < len(coords_tracked[frame])
        
        _coords_tracked = coords_tracked[frame][index_tracked_polygon]
        
        # deal with tracked polygon
        track_rois.append(_coords_tracked)
        track_maps.setdefault(f'{frame}_tracked',[]).append(_coords_tracked)
        
        if two_colour_analysis:
            # find untracked polygons with centroids contained inside tracked polygons
            untracked_centroids_this_frame = centroids_untracked.get(frame)
            tracked_path = mpltPath.Path(np.transpose(_coords_tracked))
            inside_polygon = tracked_path.contains_points(untracked_centroids_this_frame)
            contained_untracked_indices = np.where(inside_polygon)[0]
            
            if len(contained_untracked_indices)==0:
                track_maps.setdefault(f'{frame}_untracked', [])
                continue
            
            for i in contained_untracked_indices:
                _coords_untracked = coords_untracked[frame][i]
                track_maps.setdefault(f'{frame}_untracked', []).append(_coords_untracked)
        
    track_rois = np.stack(track_rois,axis=0)
    return track_rois, track_maps


def get_box_for_rois(track_rois, img_shape, pad=0):
    assert len(img_shape) == 2
    vmin = np.min(track_rois,axis=(0,2)) - pad
    vmax = np.max(track_rois,axis=(0,2)) + pad
    slices = tuple (
        slice(
            np.maximum(0, int(np.round(a))),
            np.minimum(s, int(np.round(b)))
        )
        for a,b,s in zip(vmin,vmax,img_shape)
    )
    return vmin, vmax, slices


def translate_rois(rois, vmin):
    # print(type(rois))
    if vmin.ndim == 1: vmin = np.expand_dims(vmin, -1)
    if isinstance(rois,np.ndarray):
        return np.stack(translate_rois(list(rois), vmin))
    elif isinstance(rois,(tuple,list)):
        return type(rois)(roi - vmin for roi in rois)
    elif isinstance(rois,dict):
        return {k: translate_rois(v, vmin) for k,v in rois.items()}


def _plot_polygon(x,y,score=2,color='w',ax=None,**kwargs):
    a,b = list(x),list(y)
    a += a[:1]
    b += b[:1]
    if ax is None:
        ax = plt
    ax.plot(a,b,'--', alpha=1, linewidth=score, zorder=1, color=color, **kwargs)
    

def plot_track(crop, crop_rois, preview_dir, j, n_cols=16, figsize=(25,5),  imshow_kwargs=None, poly_kwargs=None):
    if imshow_kwargs is None: imshow_kwargs = {}
    if poly_kwargs is None: poly_kwargs = {}
    n_frames = len(crop)
    n_rows = int(np.ceil(n_frames / n_cols))
    fig, ax = plt.subplots(n_rows, n_cols, figsize=figsize)
    for i,(c,a) in enumerate(zip(crop,ax.ravel())):
        a.imshow(c, **imshow_kwargs)
        a.set_title(i)
        if f'{i}_tracked' in crop_rois:
            [_plot_polygon(r[1],r[0], ax=a, color='w', **poly_kwargs) for r in crop_rois.get(f'{i}_tracked')]
        if f'{i}_untracked' in crop_rois:
            [_plot_polygon(r[1],r[0], ax=a, color='c', **poly_kwargs) for r in crop_rois.get(f'{i}_untracked')]
    [a.axis('off') for a in ax.ravel()]
    plt.tight_layout()
    save_path = preview_dir/f'{j:03}'
    plt.savefig(str(save_path))
    plt.close()    
    
    
def export_crop_with_rois(tif_dir, mask_tif_dir, polygon_dir, rois_dir, i, crop, crop_rois, two_colour_analysis, axes='TCYX'):
    crop_name = f'{f}_crop_{i:03}'
    
    crop_tif = Path(tif_dir / f'{crop_name}.tif')
    
    if not two_colour_analysis:
        crop_lbl_tif_tracked = Path(mask_tif_dir / f'{crop_name}_mask.tif')
        crop_roi_tracked = Path(rois_dir / f'{crop_name}.zip')
        crop_roi_npz_tracked = Path(polygon_dir / f'{crop_name}.npz')
    else:
        crop_lbl_tif_tracked = Path(mask_tif_dir / 'tracked channel' / f'{crop_name}_mask.tif')
        crop_roi_tracked = Path(rois_dir / 'tracked channel' / f'{crop_name}.zip')
        crop_roi_npz_tracked = Path(polygon_dir / 'tracked channel' / f'{crop_name}.npz')
        
        crop_lbl_tif_untracked = Path(mask_tif_dir / 'untracked channel' / f'{crop_name}_mask.tif')
        crop_roi_untracked = Path(rois_dir / 'untracked channel' / f'{crop_name}.zip')
        crop_roi_npz_untracked = Path(polygon_dir / 'untracked channel' / f'{crop_name}.npz')
    
    n_frames = len(crop)
    save_tiff_imagej_compatible(str(crop_tif), crop, axes, metadata=imagej_metadata)
    
    tracked_rois_list = [crop_rois.get(f'{t}_tracked',[]) for t in range(n_frames)]
    export_imagej_rois(str(crop_roi_tracked), tracked_rois_list)
    np.savez(str(crop_roi_npz_tracked),coord=tracked_rois_list)
    
    if two_colour_analysis:
        untracked_rois_list = [crop_rois.get(f'{t}_untracked',[]) for t in range(n_frames)]
        export_imagej_rois(str(crop_roi_untracked), untracked_rois_list)
        np.savez(str(crop_roi_npz_untracked),coord=untracked_rois_list)
    
    labels_tracked = []
    for frame,rois in zip(crop,tracked_rois_list):
        lbl = np.zeros(frame.shape[-2:], np.uint8)
        for roi in rois:
            rr,cc = polygon(roi[0],roi[1],lbl.shape)
            lbl[rr,cc] = 255
        labels_tracked.append(lbl)
    labels_tracked = np.stack(labels_tracked)
    save_tiff_imagej_compatible(str(crop_lbl_tif_tracked), labels_tracked, axes[0]+axes[-2:], compress=6)
    
    if two_colour_analysis:
        labels_untracked = []
        for frame,rois in zip(crop,untracked_rois_list):
            lbl = np.zeros(frame.shape[-2:], np.uint8)
            for roi in rois:
                if np.shape(roi)[-1]==0:
                    continue
                rr,cc = polygon(roi[0],roi[1],lbl.shape)
                lbl[rr,cc] = 255
            labels_untracked.append(lbl)
        labels_untracked = np.stack(labels_untracked)
        save_tiff_imagej_compatible(str(crop_lbl_tif_untracked), labels_untracked, axes[0]+axes[-2:], compress=6)
    
    
def export_crops_to_file(f, tracks, polygons_tracked, polygons_untracked, T, axes):

    # set up file saving structure
      
    crop_dir = results_dir / f"crops_{f}"
    crop_dir.mkdir(exist_ok=True)
    
    tif_dir = crop_dir / f'tifs'
    tif_dir.mkdir(exist_ok=True)
    
    mask_tif_dir = crop_dir / f'mask tifs'
    mask_tif_dir.mkdir(exist_ok=True)
    if polygons_untracked is not None:
        mask_tif_tracked_dir = mask_tif_dir / f'tracked channel'
        mask_tif_tracked_dir.mkdir(exist_ok=True)
        mask_tif_untracked_dir = mask_tif_dir / f'untracked channel'
        mask_tif_untracked_dir.mkdir(exist_ok=True)
    
    polygon_dir = crop_dir / f'polygons'
    polygon_dir.mkdir(exist_ok=True)
    if polygons_untracked is not None:
        polygon_tracked_dir = polygon_dir / f'tracked channel'
        polygon_tracked_dir.mkdir(exist_ok=True)
        polygon_untracked_dir = polygon_dir / f'untracked channel'
        polygon_untracked_dir.mkdir(exist_ok=True)
    
    rois_dir = crop_dir / f'imagej rois'
    rois_dir.mkdir(exist_ok=True)
    if polygons_untracked is not None:
        rois_tracked_dir = rois_dir / f'tracked channel'
        rois_tracked_dir.mkdir(exist_ok=True)
        rois_untracked_dir = rois_dir / f'untracked channel'
        rois_untracked_dir.mkdir(exist_ok=True)
    
    preview_dir = crop_dir / f'preview timelapse'
    preview_dir.mkdir(exist_ok=True)
    

    for i,track in tqdm(enumerate(tracks),total=len(tracks)):
        track_rois, track_maps = get_rois_for_track(track, polygons_tracked, polygons_untracked)
        vmin, vmax, slices = get_box_for_rois(track_rois, T.shape[-2:], pad=3)
        crop_T         = T[((slice(None),)*(T.ndim-2))+slices]
        crop_timelapse = timelapse[(slice(None),)+slices+(slice(None),)]

        crop_rois_per_frame = translate_rois(track_maps, vmin)
        export_crop_with_rois(tif_dir, mask_tif_dir, polygon_dir, rois_dir, i, crop_T, crop_rois_per_frame, two_colour_analysis, axes=axes)
        plot_track(crop_timelapse, crop_rois_per_frame, preview_dir, i, n_cols=16, figsize=(40,8))

    return crop_dir

# Main execution loop

In [11]:
for file in raw_files:
    
    f = file.stem
    print(f'****Analysing file: {f}****')
    
    image, rois_python_tracked, rois_imagej_tracked, rois_python_untracked, rois_imagej_untracked, rois_trackmate = (
        get_matching_files(f, drift_corrected, two_colour_analysis))
    
    timelapse, T, imagej_metadata, axes = track_pre_process(image)
    
    if not two_colour_analysis:
        polygons_tracked, polygons_untracked, tracks = load_rois_and_tracks(rois_trackmate, rois_python_tracked)
    else:
        polygons_tracked, polygons_untracked, tracks = load_rois_and_tracks(rois_trackmate, rois_python_tracked, rois_python_untracked)
    
    export_crops_to_file(f, tracks, polygons_tracked, polygons_untracked, T, axes)
    

****Analysing file: membrane_dna_1****
Normalizing each frame -> 'timelapse' is meant for plotting, use 'T' for further analysis



Timelapse has axes TYX with shape (64, 600, 600, 3)
Loading tracked python polygons from data\two-colour data\stardist results\channel 1\membrane_dna_1_prob=default_nms=0.70.npz
Loading untracked python polygons from data\two-colour data\stardist results\channel 2\membrane_dna_1_prob=default_nms=0.70.npz
Loading ROIs per track from data\two-colour data\tracking results\DRIFTCORRECTED_membrane_dna_1_tracks.csv
Found 23 tracks in dataset!



****Analysing file: membrane_dna_2****
Normalizing each frame -> 'timelapse' is meant for plotting, use 'T' for further analysis



Timelapse has axes TYX with shape (69, 600, 600, 3)
Loading tracked python polygons from data\two-colour data\stardist results\channel 1\membrane_dna_2_prob=default_nms=0.70.npz
Loading untracked python polygons from data\two-colour data\stardist results\channel 2\membrane_dna_2_prob=default_nms=0.70.npz
Loading ROIs per track from data\two-colour data\tracking results\DRIFTCORRECTED_membrane_dna_2_tracks.csv
Found 9 tracks in dataset!


# Next steps
I recommend opening all the images in the 'preview timelapse' folder as a stack in ImageJ. You can then inspect each of the identified division events and the associated segmentations drawn over the top.

## `Curation_helper.ijm`
This macro can help you generate a .csv file of manually curated tracks for further analysis.
* Drag the 'preview timelapse' folder in 'results/(dataset being analysed)' onto ImageJ. When it asks you if you want to open the images as a stack, the answer is yes yes you do.
* Press `Ctrl+A` to select the whole area.
* Make sure the ROI manager is empty.
* Move through the stack, and if the preview looks like an event that you want to analyse further, press `T` on the keyboard to add the frame to the ROI manager.
* When you've finished checking the previews, run `Curation_helper.ijm`. This will prompt you for a save directory - this should be the relevant folder in 'results' for the dataset you are measuring.
* This will save a .csv file containing the indices of the frames called `curated_results.csv` that can then be used in the `Measure_divisions_.ipynb` notebook, if you like.